In [6]:
!pip install -qqq argilla datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.0 MB/s eta 0:00:00


Reference  : https://docs.argilla.io/en/latest/guides/log_load_and_prepare_data.html


In [7]:
import argilla as rg

In [10]:
import os
import argilla as rg

rg.init(
    api_url="<API_URL>",
    api_key="admin.apikey"
)

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:153: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


In [11]:
from datasets import load_dataset

In [13]:
# https://huggingface.co/datasets/argilla/databricks-dolly-15k-curated-multilingual
data = load_dataset("argilla/databricks-dolly-15k-curated-multilingual", split="en")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating de split:   0%|          | 0/15015 [00:00<?, ? examples/s]

Generating en split:   0%|          | 0/15015 [00:00<?, ? examples/s]

Generating es split:   0%|          | 0/15015 [00:00<?, ? examples/s]

Generating fr split:   0%|          | 0/15015 [00:00<?, ? examples/s]

In [14]:
df = data.to_pandas()
df.head(10)

,instruction,context,response,category,instruction_original_en,context_original_en,response_original_en,id
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...,closed_qa,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...,0
1,Which is a species of fish? Tope or Rope,,Tope,classification,Which is a species of fish? Tope or Rope,,Tope,1
2,Why can camels survive for long without water?,,Camels use the fat in their humps to keep them...,open_qa,Why can camels survive for long without water?,,Camels use the fat in their humps to keep them...,2
3,"Alice's parents have three daughters: Amy, Jes...",,The name of the third daughter is Alice,open_qa,"Alice's parents have three daughters: Amy, Jes...",,The name of the third daughter is Alice,3
4,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981.",closed_qa,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981.",4
5,If I have more pieces at the time of stalemate...,Stalemate is a situation in chess where the pl...,No. \nStalemate is a drawn position. It doesn'...,information_extraction,If I have more pieces at the time of stalemate...,Stalemate is a situation in chess where the pl...,No. \nStalemate is a drawn position. It doesn'...,5
6,"Given a reference text about Lollapalooza, whe...",Lollapalooza /ˌlɒləpəˈluːzə/ (Lolla) is an ann...,Lollapalooze is an annual musical festival hel...,closed_qa,"Given a reference text about Lollapalooza, whe...",Lollapalooza /ˌlɒləpəˈluːzə/ (Lolla) is an ann...,Lollapalooze is an annual musical festival hel...,6
7,Who gave the UN the land in NY to build their HQ,,John D Rockerfeller,open_qa,Who gave the UN the land in NY to build their HQ,,John D Rockerfeller,7
8,Why mobile is bad for human,,We are always engaged one phone which is not g...,brainstorming,Why mobile is bad for human,,We are always engaged one phone which is not g...,8
9,Who was John Moses Browning?,"John Moses Browning (January 23, 1855 – Novemb...",John Moses Browning is one of the most well-kn...,information_extraction,Who was John Moses Browning?,"John Moses Browning (January 23, 1855 – Novemb...",John Moses Browning is one of the most well-kn...,9


In [16]:
records = []
for row in data:
  record = rg.FeedbackRecord(
      fields = {
          "category" : row["category"],
          "instruction": row["instruction"],
          "context" : row["context"],
          "response" : row["response"]
      },
      external_id = row["id"]
  )
  records.append(record)

In [17]:
fields = [
    rg.TextField(name="category", title="Task Category"),
    rg.TextField(name="instruction"),
    rg.TextField(name="context", title="Input", required=False),
    rg.TextField(name="response"),
]

In [18]:
# list of questions to display in the feedback form
questions =[
    rg.TextQuestion(
        name="new-instruction",
        title="Final instruction:",
        description="Write the final version of the instruction, making sure that it matches the task category. If the original instruction is ok, copy and paste it here.",
        required=True
    ),
    rg.TextQuestion(
        name="new-input",
        title="Final input:",
        description="Write the final version of the input, making sure that it makes sense with the task category. If the original input is ok, copy and paste it here. If an input is not needed, leave this empty.",
        required=False
    ),
    rg.TextQuestion(
        name="new-response",
        title="Final response:",
        description="Write the final version of the response, making sure that it matches the task category and makes sense for the instruction (and input) provided. If the original response is ok, copy and paste it here.",
        required=True
    )
]

guidelines = "In this dataset, you will find a collection of records that show a category, an instruction, an input and a response to that instruction. The aim of the project is to correct the instructions, intput and responses to make sure they are of the highest quality and that they match the task category that they belong to. All three texts should be clear and include real information. In addition, the response should be as complete but concise as possible.\n\nTo curate the dataset, you will need to provide an answer to the following text fields:\n\n1 - Final instruction:\nThe final version of the instruction field. You may copy it using the copy icon in the instruction field. Leave it as it is if it's ok or apply any necessary corrections. Remember to change the instruction if it doesn't represent well the task category of the record.\n\n2 - Final input:\nThe final version of the instruction field. You may copy it using the copy icon in the input field. Leave it as it is if it's ok or apply any necessary corrections. If the task category and instruction don't need of an input to be completed, leave this question blank.\n\n3 - Final response:\nThe final version of the response field. You may copy it using the copy icon in the response field. Leave it as it is if it's ok or apply any necessary corrections. Check that the response makes sense given all the fields above.\n\nYou will need to provide at least an instruction and a response for all records. If you are not sure about a record and you prefer not to provide a response, click Discard."

In [19]:
dataset = rg.FeedbackDataset(
    fields = fields,
    questions = questions,
    guidelines = guidelines
)

In [20]:
dataset.add_records(records[:5])

In [22]:
dataset.push_to_argilla(name="dolly_demo", workspace="admin")

Pushing records to Argilla...: 100%|██████████| 1/1 [00:00<00:00,  2.70it/s]
